In [1]:
import numpy
import torch
import mne
import matplotlib as plt 
import os 

# Importation & Pre-processing

In [ ]:
import os
import mne

# Chemin vers le dossier parent
dossier_parent = "C:/Users/MAREZ10/OneDrive - Université Laval/Bureau/Projet Transformers/eeg_data"

# Dictionnaire pour stocker les données MNE par sous-dossier
donnees_par_sous_dossier = {}

# Parcours des sous-dossiers
for nom_sous_dossier in os.listdir(dossier_parent):
    
    chemin_sous_dossier = os.path.join(dossier_parent, nom_sous_dossier)
    
    if os.path.isdir(chemin_sous_dossier):
        
        donnees_par_sous_dossier[nom_sous_dossier] = []
        
        for nom_fichier in os.listdir(chemin_sous_dossier):
            
            if nom_fichier.endswith('.edf'):
                
                chemin_fichier = os.path.join(chemin_sous_dossier, nom_fichier)
                donnees_mne = mne.io.read_raw_edf(chemin_fichier)
                donnees_par_sous_dossier[nom_sous_dossier].append(donnees_mne)

# Affichage des informations sur les données MNE
for nom_sous_dossier, donnees_mne in donnees_par_sous_dossier.items():
    print(f"Sujet {nom_sous_dossier} : {len(donnees_mne)} fichiers .edf")




In [ ]:
X = list(donnees_par_sous_dossier.keys())
Y = list(len(elem) for elem in donnees_par_sous_dossier.values())

# Création de l'histogramme
plt.figure(figsize=(10, 6))
plt.bar(X,Y, color='skyblue')
plt.xlabel('Sujets')
plt.ylabel('Nombre d\'enregistrements')
plt.title('Nombre d\'enregistrements par sujet')
plt.xticks(rotation=45, ha='right')  # Rotation des étiquettes sur l'axe des x pour une meilleure lisibilité
plt.tight_layout()  # Ajustement automatique du tracé pour éviter les chevauchements
plt.show()





### Temps d'échantillonnage - Nombre d'enregistrements n'ayant pas une durée de 1 heure par sujet 

In [ ]:
# Dictionnaire pour stocker le nombre de fichiers par sous-dossier
nombre_par_sous_dossier = {}

# Parcours des sous-dossiers
for nom_sous_dossier, donnees_mne_liste in donnees_par_sous_dossier.items():
    # Initialisation du compteur pour ce sous-dossier
    cpt = 0
    # Parcours des données MNE dans ce sous-dossier
    for donnees_mne in donnees_mne_liste:
        # Vérification si la durée de la donnée MNE est inférieure à la valeur donnée
        if donnees_mne.times[-1] < 3599.99609375:
            # Incrémentation du compteur
            cpt += 1
    # Stockage du nombre dans le dictionnaire
    nombre_par_sous_dossier[nom_sous_dossier] = cpt

# Affichage du nombre de fichiers qui satisfont la condition pour chaque sous-dossier
for nom_sous_dossier, nombre in nombre_par_sous_dossier.items():
    print(f"Sous-dossier {nom_sous_dossier} : {nombre} fichiers")


## Preprocessing & Labelisation

In [3]:
path = "C:/Users/MAREZ10/OneDrive - Université Laval/Bureau/Projet Transformers/eeg_data)"

In [ ]:
#import pyedflib
import numpy as np
import tqdm
import mne
from scipy.signal import welch,stft
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import euclidean

def extract_basic_features(signal):
    signal = (signal - np.mean(signal)) / np.std(signal)
    mean = np.mean(signal)
    std = np.std(signal)
    sample_entropy = np.log(np.std(np.diff(signal)))
    fuzzy_entropy = -np.log(euclidean(signal[:-1], signal[1:]) / len(signal))
    skewness = skew(signal)
    kurt = kurtosis(signal)
    return [mean, std, sample_entropy, fuzzy_entropy, skewness, kurt]

def extract_advanced_features(data, fs, window_length_sec=3):
   

    # STFT
    f, t, Zxx = stft(data, fs, nperseg=window_length_sec*fs)
    
    
    power = np.mean(np.abs(Zxx)**2, axis=1)  

    return power

def preprocess_and_extract_features_mne_with_timestamps(file_name):
    
    raw = mne.io.read_raw_edf(file_name, preload=True)

    
    raw.filter(1., 50., fir_design='firwin')

    #  EEG 
    raw.pick_types(meg=False, eeg=True, eog=False)

    window_length = 3 
    sfreq = raw.info['sfreq']  
    window_samples = int(window_length * sfreq)

   
    features_with_timestamps = []

   
    for start in range(0, len(raw.times), window_samples):
        end = start + window_samples
        if end > len(raw.times):
            break

       
        window_data, times = raw[:, start:end]
        window_data = np.squeeze(window_data)

        
        timestamp = raw.times[start]

        
        for channel_data in window_data:
            basic_features = extract_basic_features(channel_data)
            advanced_features = extract_advanced_features(channel_data, sfreq)
            combined_features = np.concatenate([[timestamp], basic_features, advanced_features])
            features_with_timestamps.append(combined_features)

    return np.array(features_with_timestamps)

preprocess_and_extract_features_mne_with_timestamps(path+"/chb01/chb01_03.edf")

In [5]:
import os

def extractTarget(summary_file_path, edf_file_path):
    
    edf_file_name = os.path.basename(edf_file_path)
    seizure_start_time = None
    seizure_end_time = None
   
    with open(summary_file_path, 'r') as file:
        lines = file.readlines()
    
    found = False
    
    for line in lines:
        if "File Name: " + edf_file_name in line:
            found = True
        if found:
            if "Number of Seizures in File: 0" in line:
                return None, None  
            if "Seizure Start Time:" in line:
                seizure_start_time = int(line.split(": ")[1].split(" ")[0])
            if "Seizure End Time:" in line:
                seizure_end_time = int(line.split(": ")[1].split(" ")[0])
                break  

    return seizure_start_time, seizure_end_time

In [11]:
import glob

def extract_data_and_labels(edf_file_path, summary_file_path):

    
    X = preprocess_and_extract_features_mne_with_timestamps(edf_file_path)
    
    seizure_start_time, seizure_end_time = extractTarget(summary_file_path, edf_file_path)
    y = np.array([1 if seizure_start_time <= row[0] <= seizure_end_time else 0 for row in X])

    #Time
    X = X[:,1:]
    return X,y


def load_data(subject_id,base_path):
   
    #edf_file_path = sorted(glob.glob(os.path.join(base_path, "chb"+subject_id+".edf"))) 
    #summary_file_path = os.path.join(base_path, "chb"+subject_id+"/chb"+subject_id+"-summary.txt")
    
    all_X = []
    all_y = []
    for edf_file_path in edf_file_path:
        X, y = extract_data_and_labels(edf_file_path, summary_file_path)
        all_X.append(X)
        all_y.append(y)
    return all_X,all_y

In [12]:
sujet_1 = load_data('03', path)

In [13]:
sujet_1

([], [])